In [35]:
import statsmodels
import pandas as pd
import numpy as np
import statsmodels
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

# 1

In [36]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [37]:
raw = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [38]:
raw.religion.describe()

count       4361
unique         4
top       spirit
freq        1841
Name: religion, dtype: object

In [39]:
raw.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

Ответ: 4 ПРАВИЛЬНО

# 2

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [40]:
#raw = raw.dropna(axis=0)

In [41]:
raw.count()


ceb         4361
age         4361
educ        4361
religion    4361
idlnchld    4241
knowmeth    4354
usemeth     4290
evermarr    4361
agefm       2079
heduc       1956
urban       4361
electric    4358
radio       4359
tv          4359
bicycle     4358
dtype: int64

Ответ: 1834 ПРАВИЛЬНО

In [42]:
raw.agefm.count()

2079

# 3

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый <b>бинарный признак</b>


<img src='import_2.jpg'>

заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=β0+β1x1+β2x2,
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
y=β0+β1x,
а там, где x1 было "не применимо", получится
y=β0+β1c+β2.
Выбор c влияет только на значение и интерпретацию β2, но не β1.



<img src="import_3.jpg">

In [43]:
#создали признак 1
raw['nevermarr'] = raw['agefm'].apply(lambda x : 1 if pd.isnull(x) else 0)


In [44]:
raw.nevermarr.count()

4361

In [45]:
#удалили 2
raw.drop('evermarr',axis=1,inplace=True)


In [46]:
raw.agefm.count()

2079

In [47]:
#заменили 3
raw['agefm'] = raw['agefm'].apply(lambda x : 0 if pd.isnull(x) else x)


In [48]:
raw.agefm.count()

4361

In [49]:
raw['heduc'].count()

1956

In [50]:
raw.nevermarr.value_counts()

1    2282
0    2079
Name: nevermarr, dtype: int64

In [51]:
raw.nevermarr.value_counts()

1    2282
0    2079
Name: nevermarr, dtype: int64

In [52]:
raw.nevermarr[raw.nevermarr==1].count()-(4361-raw.heduc.count())

-123

ответ в 3-м: 123

In [53]:
raw.nevermarr.count()-raw.heduc.count()

2405

In [54]:
raw.heduc.value_counts()

0.0     692
7.0     396
10.0    210
6.0     102
12.0     99
4.0      73
3.0      64
5.0      62
2.0      47
15.0     35
9.0      35
14.0     29
16.0     26
1.0      25
17.0     17
8.0      17
11.0      9
13.0      8
18.0      4
19.0      4
20.0      2
Name: heduc, dtype: int64

In [55]:
raw['heduc']=raw.apply(lambda row: -1 if pd.isnull(row['heduc']) and row['nevermarr']==1 else row['heduc'], axis=1)

In [56]:
raw.nevermarr.count()-raw['heduc'].count()

123

ответ: 123 правильно

# 4

<img src="import_4.jpg">

    Почему не работает:
   1)не заменили в первой части все(проверь)
   2)неправильно удалили оставшиеся NaN в 4 признаках

In [57]:
#индикаторы: idlnchld_noans, heduc_noans, usemeth_noans 
#тут их пилилм признаки

raw['idlnchld_noans'] = raw['idlnchld'].apply(lambda x : 1 if pd.isnull(x) else 0)
raw['heduc_noans'] = raw['heduc'].apply(lambda x : 1 if pd.isnull(x) else 0)
raw['usemeth_noans'] = raw['usemeth'].apply(lambda x : 1 if pd.isnull(x) else 0)

In [58]:
raw['heduc']=raw.heduc.fillna(-2.,axis=0)

In [59]:
raw['idlnchld']=raw.idlnchld.fillna(-1.,axis=0)
raw['usemeth']=raw.usemeth.fillna(-1.,axis=0)

In [60]:
raw_clear=raw.dropna(axis=0,how='any',subset=['knowmeth', 'electric', 'radio', 'tv', 'bicycle'])

In [61]:
raw_clear.size

78264

In [62]:
raw_clear.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [63]:
raw

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0
5,1,30,5,spirit,5.0,1.0,0.0,24.0,7.0,1,1.0,0.0,0.0,0.0,0,0,0,0
6,3,42,4,other,3.0,1.0,0.0,15.0,11.0,1,1.0,0.0,1.0,0.0,0,0,0,0
7,1,36,7,other,4.0,1.0,1.0,24.0,9.0,1,1.0,0.0,0.0,0.0,0,0,0,0
8,4,37,16,catholic,4.0,1.0,1.0,26.0,17.0,1,1.0,1.0,1.0,1.0,0,0,0,0
9,1,34,5,protestant,4.0,1.0,1.0,18.0,3.0,1,0.0,1.0,0.0,0.0,0,0,0,0


ответ 78264 Правильно!!


# 5

<img src='import_5.jpg'>

In [64]:
raw_clear.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'agefm', u'heduc', u'urban', u'electric', u'radio', u'tv',
       u'bicycle', u'nevermarr', u'idlnchld_noans', u'heduc_noans',
       u'usemeth_noans'],
      dtype='object')

In [65]:
import statsmodels.formula.api as smf
m1 = smf.ols('ceb ~age + educ + religion + idlnchld + knowmeth+heduc_noans+idlnchld_noans+usemeth_noans +usemeth+agefm+heduc+urban+electric+radio+tv+bicycle+nevermarr', 
             data=raw_clear)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 15 Apr 2018   Prob (F-statistic):               0.00
Time:                        20:22:42   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

ответ: 0.644 Правильно!!

# 6

<img src='import_6.jpg'>

ответ:3 ПРАВИЛЬНО

# 7

<img src='import_7.jpg'> 

In [66]:
 sms.het_breushpagan(fitted.resid, fitted.model.exog)

(1120.1205909391381,
 1.1452927633439797e-225,
 79.046224328159923,
 2.0791008841131832e-262)

ПРАВИЛЬНО

# 8

<img src='import_8.jpg'>

In [67]:
m2 = smf.ols('ceb ~age + educ + idlnchld + knowmeth+heduc_noans+idlnchld_noans+usemeth_noans +usemeth+agefm+heduc+urban+electric+bicycle+nevermarr', 
             data=raw_clear)
fitted8 = m2.fit()
print fitted8.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sun, 15 Apr 2018   Prob (F-statistic):               0.00
Time:                        20:22:42   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [68]:
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

F=0.919236, p=0.467231, k1=5.000000


ответ:0.4672 Правильно!!

# 9

<img src='import_9.jpg'>

In [69]:
m3 = smf.ols('ceb ~age + educ + idlnchld + knowmeth+heduc_noans+idlnchld_noans+agefm+heduc+urban+electric+bicycle+nevermarr', 
             data=raw_clear)
fitted9 = m3.fit()
print fitted9.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Sun, 15 Apr 2018   Prob (F-statistic):               0.00
Time:                        20:22:42   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [70]:
m2.fit().compare_f_test(m3.fit())

(92.890582301096657, 3.1552009480418772e-40, 2.0)

In [81]:
fitted9.params.idlnchld_noans+fitted9.params.idlnchld

0.59714013878459637

# 10

<img src='import_10.jpg'>